In [1]:
import numpy as np
import gp_grief
from pdb import set_trace
from scipy.stats import multivariate_normal as mvn
from numpy.testing import *

For a random dataset, check to ensure that the matrix inverse, matrix determinant and log-marginal likelihood are correct

In [2]:
np.random.seed(0)
d = 5
n = 100
x = np.random.rand(n,d)
y = np.random.rand(n,1)
grid = gp_grief.grid.InducingGrid(x)
kern = gp_grief.kern.GriefKernel(kern_list=[gp_grief.kern.GPyKernel(1,lengthscale=0.5, kernel='RBF'),]*d,
                                grid=grid, n_eigs=50)
m = gp_grief.models.GPGrief(x,y,kern,noise_var=0.1)
lml = m._compute_log_likelihood(m.parameters)
K = m._mv_cov(np.identity(n))

# check the linear system solve accuracy
alp = m._mv_cov_inv(y)
alp_exact = np.linalg.solve(K, y)
assert_array_almost_equal(alp, alp_exact, decimal=6, err_msg="linear system solve failed.")

# check the log determinant
log_det = m._cov_log_det()
log_det_exact = np.linalg.slogdet(K)[1]
assert_almost_equal(log_det, log_det_exact, decimal=6, err_msg="log determinant failed.")

# check the LML accuacy
lml_exact = mvn.logpdf(x=y.squeeze(), mean=np.zeros(n), cov=K)
assert_almost_equal(lml, lml_exact, decimal=6, err_msg="LML failed.")

print "Tests Passed!"

Tests Passed!
